In [1]:
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.5f}'.format
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision = 5, suppress = True)

In [2]:
train = pd.read_csv('datasets/KDDTrain+.csv')
test = pd.read_csv('datasets/KDDTest+.csv')

In [3]:
combined_file = pd.concat((train, test))

In [4]:
combined_file = combined_file[combined_file.duplicated() == 0]

In [5]:
combined_file = combined_file.drop('difficulty_lvl', axis = 1)

In [8]:
combined_file.attack_type.unique()

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl', 'saint', 'mscan', 'apache2',
       'snmpgetattack', 'processtable', 'httptunnel', 'ps', 'snmpguess',
       'mailbomb', 'named', 'sendmail', 'xterm', 'worm', 'xlock',
       'xsnoop', 'sqlattack', 'udpstorm'], dtype=object)

In [43]:
## removes these features to level out dataset
combined_file = combined_file.query("service != 'aol'")
combined_file = combined_file.query("service != 'harvest'")
combined_file = combined_file.query("service != 'http_2784'")
combined_file = combined_file.query("service != 'http_8001'")
combined_file = combined_file.query("service != 'red_i'")
combined_file = combined_file.query("service != 'urh_i'")
combined_file = combined_file.query("service != 'printer'")
combined_file = combined_file.query("service != 'rje'")
combined_file = combined_file.query("service != 'printer'")
combined_file = combined_file.query("service != 'rje'")
    
## changes all the attack types to simply 'attack'
combined_file.loc[combined_file.attack_type != 'normal', 'attack_type'] = 'attack'
    
## OHE protocol, service and flag features
proto_ohe = pd.get_dummies(combined_file['protocol_type'])
service_ohe = pd.get_dummies(combined_file['service'])
flag_ohe = pd.get_dummies(combined_file['flag'])
    
combined_file = combined_file.drop(['protocol_type', 'service', 'flag'], axis = 1)
combined_file = pd.concat([combined_file, proto_ohe, service_ohe, flag_ohe], axis = 1)
    
## drop the 'attack_type' column so it does not get normalises
combined_file_labels = combined_file['attack_type']
combined_file = combined_file.drop(['attack_type'], axis = 1)
combined_file = 255 * combined_file / combined_file.max(numeric_only = True)

## makes sure there are no NaNs
for column in combined_file:
    if combined_file[column].isna().any():
        combined_file[column] = float(0)
    
combined_file['attack_type'] = combined_file_labels

In [49]:
combined_file

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,icmp,tcp,udp,IRC,X11,Z39_50,auth,bgp,courier,csnet_ns,ctf,daytime,discard,domain,domain_u,echo,eco_i,ecr_i,efs,exec,finger,ftp,ftp_data,gopher,hostnames,http,http_443,imap4,iso_tsap,klogin,kshell,ldap,link,login,mtp,name,netbios_dgm,netbios_ns,netbios_ssn,netstat,nnsp,nntp,ntp_u,other,pm_dump,pop_2,pop_3,private,remote_job,shell,smtp,sql_net,ssh,sunrpc,supdup,systat,telnet,tftp_u,tim_i,time,urp_i,uucp,uucp_path,vmnet,whois,OTH,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH,attack_type
0,0.00000,0.00009,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.99804,0.99804,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,150.00000,25.00000,43.35000,7.65000,43.35000,0.00000,0.00000,0.00000,12.75000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
1,0.00000,0.00003,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,6.48728,0.49902,0.00000,0.00000,0.00000,0.00000,20.40000,38.25000,0.00000,255.00000,1.00000,0.00000,153.00000,224.40000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
2,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,61.37965,2.99413,255.00000,255.00000,0.00000,0.00000,12.75000,17.85000,0.00000,255.00000,26.00000,25.50000,12.75000,0.00000,0.00000,255.00000,255.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,attack
3,0.00000,0.00004,0.00159,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.

In [50]:
combined_file.to_csv('combined_dataset.csv', index = 0)